# Sprint1課題 機械学習フロー

## 【問題1】クロスバリデーション
事前学習期間は検証用データを分割しておき、それに対して指標値を計算することで検証を行っていました。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。

具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。

[sklearn.model_selection.KFold — scikit-learn 0.20.2 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)

### データ読み込み

In [101]:
import pandas as pd
import numpy as np

# read train data
df = pd.read_csv('../../../../diveintocode-ml/week4/application_train.csv')
df_test = pd.read_csv('../../../../diveintocode-ml/week4/application_test.csv')

In [102]:
#Confirm data
print(df.shape)
df.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### One-hot Encoding

In [103]:
#カテゴリカル変数をOneHotEncodingを行う
df_dummies = pd.get_dummies(df)
df_dummies_test = pd.get_dummies(df_test)
df_dummies.head()


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,0,0,0,0,0


In [105]:
df_dummies_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018850,-19241,-2329,-5170.0,...,0,0,0,0,0,0,1,0,1,0
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035792,-18064,-4469,-9118.0,...,0,0,0,0,0,0,0,0,0,0
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019101,-20038,-4458,-2175.0,...,0,0,0,0,0,0,0,0,0,0
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,-13976,-1866,-2000.0,...,0,0,0,0,0,1,0,0,1,0
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010032,-13040,-2191,-4000.0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
#trainの欠損値を確認
print(df_dummies.isnull().sum())

SK_ID_CURR                                       0
TARGET                                           0
CNT_CHILDREN                                     0
AMT_INCOME_TOTAL                                 0
AMT_CREDIT                                       0
AMT_ANNUITY                                     12
AMT_GOODS_PRICE                                278
REGION_POPULATION_RELATIVE                       0
DAYS_BIRTH                                       0
DAYS_EMPLOYED                                    0
DAYS_REGISTRATION                                0
DAYS_ID_PUBLISH                                  0
OWN_CAR_AGE                                 202929
FLAG_MOBIL                                       0
FLAG_EMP_PHONE                                   0
FLAG_WORK_PHONE                                  0
FLAG_CONT_MOBILE                                 0
FLAG_PHONE                                       0
FLAG_EMAIL                                       0
CNT_FAM_MEMBERS                

In [108]:
# 欠損値を補完
df_dummies = df_dummies.fillna(df_dummies.mean())
df_dummies_test = df_dummies_test.fillna(df_dummies.mean())

In [109]:
#欠損がなくなっていることを確認
print(df_dummies.isnull().sum().sum())
print(df_dummies_test.isnull().sum().sum())

0
0


### 特徴量の抽出

In [8]:
# trainデータのTARGETとfeatures間の相関係数を算出
feature_corr = df_dummies.corr(method='pearson')
feature_corr

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
SK_ID_CURR,1.000000,-0.002108,-0.001129,-0.001820,-0.000343,-0.000433,-0.000232,0.000849,-0.001500,0.001366,...,0.000553,0.002073,-0.000976,0.000072,-0.001396,0.002300,-1.280782e-03,-0.000270,0.000510,0.002549
TARGET,-0.002108,1.000000,0.019187,-0.003982,-0.030369,-0.012817,-0.039628,-0.037227,0.078239,-0.044932,...,0.000982,-0.006777,-0.001713,-0.009384,0.000628,-0.033119,-1.265666e-02,0.007946,-0.042201,0.004829
CNT_CHILDREN,-0.001129,0.019187,1.000000,0.012882,0.002145,0.021374,-0.001826,-0.025573,0.330938,-0.239818,...,-0.001830,-0.005272,-0.000709,0.001607,-0.002032,-0.020892,-2.508821e-02,0.011036,-0.038644,0.004525
AMT_INCOME_TOTAL,-0.001820,-0.003982,0.012882,1.000000,0.156870,0.191657,0.159600,0.074796,0.027261,-0.064223,...,0.000718,0.011696,0.006149,0.023886,0.003886,0.032753,1.652333e-02,-0.003369,0.050174,-0.002894
AMT_CREDIT,-0.000343,-0.030369,0.002145,0.156870,1.000000,0.770127,0.986588,0.099738,-0.055436,-0.066838,...,0.000869,0.014314,0.007987,0.027255,0.005799,0.046644,9.756049e-03,-0.007373,0.058256,-0.004308
AMT_ANNUITY,-0.000433,-0.012817,0.021374,0.191657,0.770127,1.000000,0.774661,0.118424,0.009445,-0.104331,...,0.002166,0.015589,0.010077,0.035318,0.005968,0.054987,9.188486e-03,-0.007711,0.068144,-0.003910
AMT_GOODS_PRICE,-0.000232,-0.039628,-0.001826,0.159600,0.986588,0.774661,1.000000,0.103482,-0.053416,-0.064826,...,0.000796,0.013687,0.009953,0.028299,0.005877,0.049740,1.288238e-02,-0.009128,0.062849,-0.004846
REGION_POPULATION_RELATIVE,0.000849,-0.037227,-0.025573,0.074796,0.099738,0.118424,0.103482,1.000000,-0.029582,-0.003980,...,0.011212,0.078062,0.016288,0.045775,0.013435,0.092652,1.078225e-02,-0.056263,0.103513,-0.029932
DAYS_BIRTH,-0.001500,0.078239,0.330938,0.027261,-0.055436,0.009445,-0.053416,-0.029582,1.000000,-0.615864,...,0.000388,-0.006749,-0.000262,0.004421,-0.000690,-0.013597,-1.096440e-02,0.008758,-0.019852,0.005434
DAYS_EMPLOYED,0.001366,-0.044932,-0.239818,-0.064223,-0.066838,-0.104331,-0.064826,-0.003980,-0.615864,1.000000,...,-0.001761,0.003691,-0.003692,-0.006892,-0.003120,-0.004193,-6.326739e-03,-0.002846,-0.010907,-0.003571


In [9]:
# 相関係数行列から目的変数の列を抽出
feature_target_corr = feature_corr.loc[:,'TARGET']
feature_target_corr

SK_ID_CURR                                 -0.002108
TARGET                                      1.000000
CNT_CHILDREN                                0.019187
AMT_INCOME_TOTAL                           -0.003982
AMT_CREDIT                                 -0.030369
AMT_ANNUITY                                -0.012817
AMT_GOODS_PRICE                            -0.039628
REGION_POPULATION_RELATIVE                 -0.037227
DAYS_BIRTH                                  0.078239
DAYS_EMPLOYED                              -0.044932
DAYS_REGISTRATION                           0.041975
DAYS_ID_PUBLISH                             0.051457
OWN_CAR_AGE                                 0.020871
FLAG_MOBIL                                  0.000534
FLAG_EMP_PHONE                              0.045982
FLAG_WORK_PHONE                             0.028524
FLAG_CONT_MOBILE                            0.000370
FLAG_PHONE                                 -0.023806
FLAG_EMAIL                                 -0.

In [11]:
#相関係数の絶対値が大きい順にSort
feature_target_corr = abs(feature_target_corr).sort_values(ascending=False)
feature_target_corr

TARGET                                               1.000000
EXT_SOURCE_2                                         0.160303
EXT_SOURCE_3                                         0.157397
EXT_SOURCE_1                                         0.099152
DAYS_BIRTH                                           0.078239
REGION_RATING_CLIENT_W_CITY                          0.060893
REGION_RATING_CLIENT                                 0.058899
NAME_INCOME_TYPE_Working                             0.057481
NAME_EDUCATION_TYPE_Higher education                 0.056593
DAYS_LAST_PHONE_CHANGE                               0.055218
CODE_GENDER_M                                        0.054713
CODE_GENDER_F                                        0.054704
DAYS_ID_PUBLISH                                      0.051457
REG_CITY_NOT_WORK_CITY                               0.050994
NAME_EDUCATION_TYPE_Secondary / secondary special    0.049824
NAME_INCOME_TYPE_Pensioner                           0.046209
ORGANIZA

In [79]:
# df_dummiesからTARGETと相関係数の高い特徴量のみを抽出
df_selected = df_dummies.loc[:, feature_target_corr[:4].index]
#確認
df_selected.head()

,TARGET,EXT_SOURCE_2,EXT_SOURCE_3,EXT_SOURCE_1
0,1,0.262949,0.139376,0.083037
1,0,0.622246,0.510853,0.311267
2,0,0.555912,0.729567,0.502130
3,0,0.650442,0.510853,0.502130
4,0,0.322738,0.510853,0.502130


In [120]:

feature_target_corr[1:4]

EXT_SOURCE_2    0.160303
EXT_SOURCE_3    0.157397
EXT_SOURCE_1    0.099152
Name: TARGET, dtype: float64

In [125]:
#test dataも同様の操作
X_test = df_dummies_test.loc[:,feature_target_corr[1:4].index]
X_test = pd.concat([df_dummies_test['SK_ID_CURR'], X_test], axis=1)
X_test.head()

,SK_ID_CURR,EXT_SOURCE_2,EXT_SOURCE_3,EXT_SOURCE_1
0,100001,0.789654,0.159520,0.752614
1,100005,0.291656,0.432962,0.564990
2,100013,0.699787,0.610991,0.502130
3,100028,0.509677,0.612704,0.525734
4,100038,0.425687,0.510853,0.202145


### 学習、推定、評価

In [14]:
# Pipeline クラスをインポート
from sklearn.pipeline import Pipeline

#標準化のために StanardScaler
from sklearn.preprocessing import StandardScaler

#Cross validationのためにStratifiedKFold
from sklearn.model_selection import StratifiedKFold

#評価用のライブラリ
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

#時間を計測するためにtime関数を取得
import time


#Cross validationを試すためにfit & predict & evaluateのための関数を用意

def fit_to_predict(model, X_train, X_test, y_train, y_test):
    """
    Parameters
    --------------
    model: learnign machine or pipeline
    X_train: train data of features with shape(n_train_samples, n_features)
    X_test:  test data of features with shape(n_test_samples, n_features)
    y_train: train data of target with shape(n_train_samples, 1)
    y_test:  test data of target with shape(n_train_samples, 1)
       
    Returns
    --------------
    
    Note
    --------------
    fit data with train samples to calculate and print AUC for test data
    
    """
    #train with train data
    model.fit(X_train, y_train)
    
    #Estimate test value
    pred = model.predict(X_test)
    prob = model.predict_proba(X_test)
    
    #AUC
    auc = roc_auc_score(y_test, prob[:, 1])    
    print('AUC: ', auc)

In [15]:
#Cross validation

# 機械学習モデルをインポート
from sklearn.neighbors import KNeighborsClassifier

#パイプライン作成
#KNN
pl = Pipeline([
   ('scl', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

#train dataとvalidation dataのためのindexを生成
s_kfold = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

#開始時刻を記録
start = time.time()

#分割数分だけcross validationしながらfit
for train_index, test_index in s_kfold.split(df_selected, df.loc[:, 'TARGET']):
    
    #trainとvalidation dataとに分ける
    X_train = df_selected.loc[train_index, df_selected.columns != 'TARGET']
    X_test = df_selected.loc[test_index, df_selected.columns != 'TARGET']
    y_train = df_selected.loc[train_index, 'TARGET']
    y_test = df_selected.loc[test_index, 'TARGET']
    
    #fit, predict, evaluate
    fit_to_predict(pl, X_train, X_test, y_train, y_test)
    
    #かかった時間を表示
    lap = time.time()#lap time
    print('lap time: ', lap-start, 'sec')

AUC:  0.592699145174736
lap time:  50.2492401599884 sec
AUC:  0.5998464640281107
lap time:  94.64048218727112 sec
AUC:  0.5973590662165805
lap time:  137.29534602165222 sec
AUC:  0.594501124405141
lap time:  181.72743105888367 sec
AUC:  0.6000059029689065
lap time:  222.28122925758362 sec


## 【問題2】グリッドサーチ
これまで分類器のパラメータは基本的にデフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになりますが、パラメータは状況に応じて最適なものを選ぶ必要があります。パラメータを探索するために グリッドサーチ と呼ばれる総当たり的手法が一般的に利用されます。

グリッドサーチをパイプラインの中に組み込みましょう。

In [86]:
# Pipeline クラスをインポート
from sklearn.pipeline import Pipeline

#Cross validationのためにStratifiedKFold
from sklearn.model_selection import StratifiedKFold

#評価用のライブラリ
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

import warnings

#時間を計測するためにtime関数を取得
import time


#Grid Searchを試すためにfit & predict & evaluateをcross validationで実行するための関数を用意

def cross_val_fit_to_predict(model, X, y, n_folds):
    """
    Parameters
    -----
    model : classification model
    X : DataFrame
    y : DataFrame
    n_folds : int
        number of folds
    
    Notes
    -----
    Stratified KFoldを行い、ロジスティック回帰を行う。
    また、グリッドサーチを用いることで、正則化項の適切なパラメータを探し出す。
    
    """
    warnings.filterwarnings('ignore')
    s_kfold = StratifiedKFold(n_splits=n_folds, random_state=None, shuffle=False)
    

    start= time.time()#start time
    
    for train_index, test_index in s_kfold.split(df_selected.loc[:, df_selected.columns != 'TARGET'], df_selected.loc[:, 'TARGET']):
        X_train = df_selected.loc[train_index, df_selected.columns != 'TARGET']
        X_test = df_selected.loc[test_index, df_selected.columns != 'TARGET']
        y_train = df_selected.loc[train_index, 'TARGET']
        y_test = df_selected.loc[test_index, 'TARGET']
                
        #学習
        model.fit(X_train, y_train)
        
        #推定
        pred = model.predict(X_test)
        prob = model.predict_proba(X_test)
        
        #評価
        auc_score = roc_auc_score(y_test, prob[:, 1])        
        best_params = model.best_params_
        
        lap= time.time()#lap time
        
        print('-------------------------')
        print('AUC: ', auc_score)
        print('Best Params: ', best_params)
        print('Processed Time: ', lap-start, 'sec')
        print('-------------------------\n')

In [24]:
#Grid searchの実行
#Decision Tree Classifier

from sklearn.model_selection import GridSearchCV
#標準化のために StanardScaler
from sklearn.preprocessing import StandardScaler
# 機械学習モデルをインポート
from sklearn.tree import DecisionTreeClassifier

parameters = {
      'dtc__criterion': ['gini', 'entropy'],
      'dtc__max_depth': [1, 5, 10, 20] 
}

pl = Pipeline([
   ('scl', StandardScaler()),
    ('dtc', DecisionTreeClassifier())
])

clf = GridSearchCV(
    estimator=pl,
    param_grid=parameters
)

X = df_selected.loc[:, df_selected.columns != 'TARGET']
y = df_selected.loc[:, df_selected.columns == 'TARGET']

cross_val_fit_to_predict(clf, X, y, 5)

-------------------------
AUC:  0.5844538747781216
Best Params:  {'dtc__criterion': 'gini', 'dtc__max_depth': 1}
Processed Time:  19.082921981811523 sec
-------------------------

-------------------------
AUC:  0.5862500011355289
Best Params:  {'dtc__criterion': 'gini', 'dtc__max_depth': 1}
Processed Time:  42.069490909576416 sec
-------------------------

-------------------------
AUC:  0.5848118872185244
Best Params:  {'dtc__criterion': 'gini', 'dtc__max_depth': 1}
Processed Time:  60.51838994026184 sec
-------------------------

-------------------------
AUC:  0.7087129352911882
Best Params:  {'dtc__criterion': 'entropy', 'dtc__max_depth': 5}
Processed Time:  75.38178300857544 sec
-------------------------

-------------------------
AUC:  0.7114999916015393
Best Params:  {'dtc__criterion': 'entropy', 'dtc__max_depth': 5}
Processed Time:  90.10481286048889 sec
-------------------------



In [22]:
#Grid searchの実行
#Random Forest Classifier

from sklearn.model_selection import GridSearchCV
#標準化のために StanardScaler
from sklearn.preprocessing import StandardScaler
# 機械学習モデルをインポート
from sklearn.ensemble import RandomForestClassifier

parameters = {
    "rfc__max_depth": [1,5, 10, 20],
#    "rfc__n_estimators":[50,100,200,300,400,500],
#    "rfc__max_features": [1, 3, 10],
#    "rfc__min_samples_split": [2, 3, 10],
#    "rfc__min_samples_leaf": [1, 3, 10],
#    "rfc__bootstrap": [True, False],
    "rfc__criterion": ["gini", "entropy"]
}

pl = Pipeline([
    ('scl', StandardScaler()),
    ('rfc', RandomForestClassifier())
])

clf = GridSearchCV(
    estimator=pl,
    param_grid=parameters
)


X = df_selected.loc[:, df_selected.columns != 'TARGET']
y = df_selected.loc[:, df_selected.columns == 'TARGET']

cross_val_fit_to_predict(clf, X, y, 5)


-------------------------
AUC:  0.7152744188982576
Best Params:  {'rfc__criterion': 'gini', 'rfc__max_depth': 5}
Processed Time:  59.05389618873596 sec
-------------------------

-------------------------
AUC:  0.7181184683822718
Best Params:  {'rfc__criterion': 'entropy', 'rfc__max_depth': 5}
Processed Time:  119.19800615310669 sec
-------------------------

-------------------------
AUC:  0.6603186719723563
Best Params:  {'rfc__criterion': 'gini', 'rfc__max_depth': 1}
Processed Time:  168.1956810951233 sec
-------------------------

-------------------------
AUC:  0.6698087756912963
Best Params:  {'rfc__criterion': 'gini', 'rfc__max_depth': 1}
Processed Time:  214.70575308799744 sec
-------------------------

-------------------------
AUC:  0.7206252583550833
Best Params:  {'rfc__criterion': 'entropy', 'rfc__max_depth': 5}
Processed Time:  272.54182410240173 sec
-------------------------



In [87]:
#Grid searchの実行
#Logistic Regression

from sklearn.model_selection import GridSearchCV
#標準化のために StanardScaler
from sklearn.preprocessing import StandardScaler
# 機械学習モデルをインポート
from sklearn.linear_model import LogisticRegression

#Grid searchで振るparameter
parameters = {
    "lgr__C": [0.001, 0.01, 0.1, 1, 10 ,100, 1000]
}

#pipeline
pl = Pipeline([
    ('scl', StandardScaler()),
    ('lgr', LogisticRegression())
])

#Grid search
clf = GridSearchCV(
    estimator=pl,
    param_grid=parameters
)

X = df_selected.loc[:, df_selected.columns != 'TARGET']
y = df_selected.loc[:, df_selected.columns == 'TARGET']

cross_val_fit_to_predict(clf, X, y, 5)


-------------------------
AUC:  0.7183678084488052
Best Params:  {'lgr__C': 0.001}
Processed Time:  8.372331142425537 sec
-------------------------

-------------------------
AUC:  0.7185508688701769
Best Params:  {'lgr__C': 0.001}
Processed Time:  16.51023006439209 sec
-------------------------

-------------------------
AUC:  0.7137798984529038
Best Params:  {'lgr__C': 0.001}
Processed Time:  24.062598943710327 sec
-------------------------

-------------------------
AUC:  0.7216565590347388
Best Params:  {'lgr__C': 0.001}
Processed Time:  30.916542053222656 sec
-------------------------

-------------------------
AUC:  0.7223899040635743
Best Params:  {'lgr__C': 0.001}
Processed Time:  39.54863905906677 sec
-------------------------



### Predictして提出
Logistic Regressionの結果を提出してみる

In [132]:
pred = clf.predict(np.array(X_test.iloc[:, 1:4]))
proba = clf.predict_proba(np.array(X_test.iloc[:, 1:4]))


In [134]:
my_submission = pd.DataFrame({
    'SK_ID_CURR': X_test.SK_ID_CURR, 'TARGET': proba[:, 1]
})

In [135]:
my_submission.to_csv('./submission.csv', index=False)

Score: 0.70918

## 【問題3】Kernelからの調査
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。そして、効果があると考えられるものを検証してください。

* 古典的なモデル以外のものを試す
* 複数のモデルでpredictした際のprobabilityを混ぜる

## 【問題4】高い汎化性能のモデル
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。

In [136]:
# 古典的なモデル以外のものを試す(lightgbm)

#Grid searchの実行

from sklearn.model_selection import GridSearchCV
#標準化のために StanardScaler
from sklearn.preprocessing import StandardScaler
# 機械学習モデルをインポート
import lightgbm as lgb


parameters = {
    "lgbm__num_leaves": [5, 10, 15, 20],
    "lgbm__min_data_in_leaf": [5, 10, 15, 20]
}

pl = Pipeline([
    ('scl', StandardScaler()),
    ('lgbm', lgb.LGBMClassifier())
])

clf = GridSearchCV(
    estimator=pl,
    param_grid=parameters
)


X = df_selected.loc[:, df_selected.columns != 'TARGET']
y = df_selected.loc[:, df_selected.columns == 'TARGET']

cross_val_fit_to_predict(clf, X, y, 5)


-------------------------
AUC:  0.721983610413508
Best Params:  {'lgbm__min_data_in_leaf': 15, 'lgbm__num_leaves': 20}
Processed Time:  98.73757910728455 sec
-------------------------

-------------------------
AUC:  0.7230317886987928
Best Params:  {'lgbm__min_data_in_leaf': 15, 'lgbm__num_leaves': 5}
Processed Time:  179.67281317710876 sec
-------------------------

-------------------------
AUC:  0.7173474042014854
Best Params:  {'lgbm__min_data_in_leaf': 5, 'lgbm__num_leaves': 10}
Processed Time:  272.43103313446045 sec
-------------------------

-------------------------
AUC:  0.7263518720578336
Best Params:  {'lgbm__min_data_in_leaf': 15, 'lgbm__num_leaves': 5}
Processed Time:  354.56075716018677 sec
-------------------------

-------------------------
AUC:  0.7272740176869265
Best Params:  {'lgbm__min_data_in_leaf': 20, 'lgbm__num_leaves': 5}
Processed Time:  473.3944420814514 sec
-------------------------



In [137]:
#Test data prediction
pred = clf.predict(np.array(X_test.iloc[:, 1:4]))
proba = clf.predict_proba(np.array(X_test.iloc[:, 1:4]))

In [138]:
my_submission = pd.DataFrame({
    'SK_ID_CURR': X_test.SK_ID_CURR, 'TARGET': proba[:, 1]
})

In [139]:
my_submission.to_csv('./submission_lgbm.csv', index=False)

Score: 0.71415

In [140]:
#複数のモデルでpredictした際のprobabilityを混ぜる
lgbm_result = pd.read_csv('./submission_lgbm.csv')
lr_result = pd.read_csv('./submission.csv')

new_result_1 = (8 * lgbm_result['TARGET'] + 2* lr_result['TARGET'])/10
new_result_2 = (2 * lgbm_result['TARGET'] + 8* lr_result['TARGET'])/10


In [143]:
np.array(new_result_1)

array([0.04773414, 0.0923364 , 0.03670349, ..., 0.05260306, 0.06437401,
       0.13162235])

In [144]:
my_submission = pd.DataFrame({
    'SK_ID_CURR': X_test.SK_ID_CURR, 'TARGET': np.array(new_result_1)
})
my_submission.to_csv('./submission_mix_1.csv', index=False)

Score: 0.71419<br>

In [145]:
my_submission = pd.DataFrame({
    'SK_ID_CURR': X_test.SK_ID_CURR, 'TARGET': np.array(new_result_2)
})
my_submission.to_csv('./submission_mix_2.csv', index=False)

Score: 0.71121
LGBMとLogistic Regressionを8:2で混ぜた平均を使った結果、単体の時よりも少し上がることがわかった。